In [4]:
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, calculate_pnl, process_and_save_address
'''
data_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁
添加了pnl的计算，我们这里只计算第一种交易的pnl，其余的暂时忽略不计。
最后对于address也进行一个处理
'''
ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
START_DATE = "2024-12-06 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2024-12-12 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE"}

In [5]:
transfers_df = fetch_and_save_erc20_transfers(
        address=ADDRESS,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        offset=1000,
    )
transfers_df = pd.read_csv(OUTPUT_FILE + "/all_erc20_transfers.csv")
record_df_1, record_df_2, record_df_3 = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, BASE_TOKENS)
# 这里我为了显示简介把process_transactions中print结果的注释掉了 如果需要可以取消注释

unique_tokens_count = record_df_1["token_symbol"].nunique()
unique_tokens = record_df_1["token_symbol"].unique()
results = []

for token in unique_tokens:
    token_df = record_df_1[record_df_1["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
    print(f"{token}: {len(token_df)} records")

    pnl_df = calculate_pnl(token_df)
    print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
    total_pnl = pnl_df["pnl"].sum()

    results.append({"token": token, "total_pnl": total_pnl})

pnl_summary_df = pd.DataFrame(results)

print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))

# BTC_record_df = record_df_1[record_df_1["formatted_record"].str.contains(r"\bcbBTC\b", regex=True)]
# # print(BTC_record_df['formatted_record'])
# pnl_df = calculate_pnl(BTC_record_df)
# print(f"Total PnL: {pnl_df['pnl'].sum()}")


2025-01-08 17:52:46,031 - INFO - Start block: 21339732, End block: 21382703
2025-01-08 17:52:46,031 - INFO - Fetching ERC20 transfers from block 21339732 to 21382703...
2025-01-08 17:52:47,135 - INFO - Page 1: Retrieved 119 transactions.
2025-01-08 17:52:48,009 - WARNING - API request failed on page 2: Request failed: No transactions found
2025-01-08 17:52:48,011 - INFO - Final dataset contains 119 unique transactions.
2025-01-08 17:52:48,014 - INFO - Data successfully saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result\erc20_transfers.csv
2025-01-08 17:52:48,015 - INFO - Transfers saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result


Formatted Transactions: part 1
Formatted Transactions: part 2
Formatted Transactions: final part
cbBTC: 38 records
+---------------------+-------------+--------+----------+----------+--------+
| Date                |   timeStamp | Type   |   amount |    price |    pnl |
+=====================+=============+========+==========+==========+========+
| 2024-11-29 17:38:35 |  1732901915 | BUY    |     1.71 | 97182.48 |  12.07 |
+---------------------+-------------+--------+----------+----------+--------+
| 2024-11-29 17:38:59 |  1732901939 | BUY    |     3.77 | 97180.47 |  34.20 |
+---------------------+-------------+--------+----------+----------+--------+
| 2024-11-29 17:39:23 |  1732901963 | BUY    |     1.71 | 97195.96 | -10.99 |
+---------------------+-------------+--------+----------+----------+--------+
| 2024-11-29 17:41:35 |  1732902095 | BUY    |     1.71 | 97193.47 |  -6.71 |
+---------------------+-------------+--------+----------+----------+--------+
| 2024-11-29 17:42:47 |  17

In [6]:
TOP_ACCOUNTS_DF = pd.read_csv(OUTPUT_FILE + "/top_accounts.csv")
transfers_df = pd.read_csv(OUTPUT_FILE + "/all_erc20_transfers.csv")
output_file = "address_counts_with_name_tags.csv"

process_and_save_address(transfers_df, TOP_ACCOUNTS_DF, output_file)

100%|██████████| 37/37 [00:31<00:00,  1.18it/s]

Data has been save to address_counts_with_name_tags.csv


In [ ]:
transfers_df = 
address_threshold = 10
important_addresses = address_counts[address_counts["total_count"] > address_threshold]
important_addresses = important_addresses.sort_values(by="total_count", ascending=False)
important_addresses = important_addresses[important_addresses["address"] != ADDRESS.lower()]
important_addresses["name"] = important_addresses["address"].apply(get_public_name_tag)

print(important_addresses[["address", "name"]])

# 测试代码   
address = "0x119ca458c5f91d3815e5f8a29633d709569b4eb3"
name_tag = get_public_name_tag(address)
print(f"Public Name Tag for {address}: {name_tag}")    

NameError: name 'address_counts' is not defined